In [22]:
import pandas as pd
import torch
import torch.nn.functional as F
import numpy as np
from data_loader import get_data

In [23]:
class model(torch.nn.Module):
    def __init__(self):
        super(model, self).__init__()
        self.fc1 = torch.nn.Linear(3000, 256)
        self.fc2 = torch.nn.Linear(256, 128)
        self.fc5 = torch.nn.LazyLinear(128)
        self.fc3 = torch.nn.Linear(128, 64)
        self.fc4 = torch.nn.Linear(64, 1)

    def forward(self, x):
        x = (self.fc1(x))
        x = (self.fc2(x))
        x = (self.fc5(x))
        x = (self.fc5(x))
        x = (self.fc5(x))
        x = (self.fc3(x))
        x = (self.fc4(x))
        return torch.sigmoid(x)

In [24]:
Model = model()

C:\Users\marto\anaconda3_2\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [25]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.ASGD(Model.parameters(), lr=0.0001)

In [26]:
train_load, test_load = get_data("data_2/emails.csv", 16)

In [27]:
for epoch in range(10):
    accuracy = 0
    ll = []
    for batch in train_load:
        criterion.zero_grad()
        x_batch, y_batch = batch
        
        y_pred = Model.forward(x_batch.to(torch.float32))

        y_pred_rounded = torch.round(y_pred)
        accuracy += (y_pred_rounded == y_batch.reshape(len(y_pred), 1)).sum().item()
        accuracy = accuracy / 16
        
        loss = criterion(y_pred, y_batch.float().reshape(len(y_pred), 1))
        ll += [loss.item()]
        loss.backward()
        
        optimizer.step()
    accuracy = accuracy / len(train_load)
    if epoch % 1 == 0:
        print(f"Epoch: {epoch}, Loss: {np.mean(ll)}, Accuracy: {accuracy}")

Epoch: 0, Loss: 0.23377422484997157, Accuracy: 0.0028448352673465334
Epoch: 1, Loss: 0.19051819367147263, Accuracy: 0.002183876138607861
Epoch: 2, Loss: 0.12103545890975398, Accuracy: 0.0024141139803907064
Epoch: 3, Loss: 0.070028301448569, Accuracy: 0.002670465044418111
Epoch: 4, Loss: 0.06470634576154083, Accuracy: 0.0026554419123897234
Epoch: 5, Loss: 0.036451578189951865, Accuracy: 0.0029108312600044216
Epoch: 6, Loss: 0.03844641254963416, Accuracy: 0.0026705235137361124
Epoch: 7, Loss: 0.017309905382228256, Accuracy: 0.0029118404118404118
Epoch: 8, Loss: 0.01635352142838947, Accuracy: 0.0029117812674629433
Epoch: 9, Loss: 0.014336891282979198, Accuracy: 0.0029118404118404118


In [29]:
nb_win = 0
nb_all = 0

for test_batch in test_load:
    x_batch, y_batch = test_batch

    y_pred = Model.forward(x_batch.to(torch.float32))

    nb_all += 1

    if y_pred == y_batch:
        nb_win += 1

print("Accuracy: {}%".format(round((nb_win / nb_all) * 100), 2))

Accuracy: 90%
